In [1]:
import sys
import logging

import numpy as np
import scipy as sp

%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
df = pd.read_csv("../train_features.csv")
df.drop("Time", axis=1, inplace=True)
df.head()


,pid,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,34.0,NaN,NaN,12.0,NaN,36.0,8.7,24.0,-2.0,16.0,NaN,NaN,6.3,NaN,45.0,NaN,NaN,NaN,NaN,NaN,84.0,1.2,3.8,61.0,NaN,NaN,100.0,NaN,114.0,24.6,94.0,NaN,NaN,142.0,7.33
1,1,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,-2.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,81.0,NaN,NaN,62.5,NaN,NaN,100.0,NaN,NaN,NaN,99.0,NaN,NaN,125.0,7.33
2,1,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,0.0,18.0,NaN,NaN,NaN,NaN,43.0,NaN,0.4,NaN,NaN,NaN,74.0,NaN,NaN,59.0,NaN,NaN,100.0,NaN,NaN,NaN,92.0,NaN,NaN,110.0,7.37
3,1,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,0.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,NaN,NaN,49.5,NaN,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,104.0,7.37
4,1,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,1.8,NaN,48.0,NaN,NaN,100.0,NaN,NaN,22.4,81.0,NaN,NaN,100.0,7.41


In [3]:
df.describe()


,pid,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,RRate,Fibrinogen,Phosphate,WBC,Creatinine,PaCO2,AST,FiO2,Platelets,SaO2,Glucose,ABPm,Magnesium,Potassium,ABPd,Calcium,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
count,227940.000000,227940.000000,9783.000000,10299.000000,20105.000000,10756.000000,81115.000000,22295.000000,12559.00000,19887.000000,187785.000000,2493.000000,11590.000000,19083.000000,17792.000000,21043.000000,5761.000000,26602.000000,18035.000000,13014.000000,47036.000000,195889.000000,17523.000000,28393.000000,152418.000000,17830.000000,5708.000000,195192.000000,719.000000,13917.000000,27297.000000,200128.000000,5326.000000,3776.000000,191650.000000,25046.000000
mean,15788.831219,62.073809,32.883114,40.091310,23.192664,2.859716,36.852136,10.628208,23.48810,-1.239284,18.154043,262.496911,3.612519,11.738649,1.495777,41.115696,193.444888,0.701666,204.666426,93.010527,142.169407,82.117276,2.004149,4.152729,64.014711,7.161149,97.796163,97.663449,1.390723,106.260185,31.283090,84.522371,1.640941,7.269240,122.369877,7.367231
std,9151.896286,16.451854,7.802065,26.034961,20.024289,2.428368,0.875152,2.074859,4.40378,4.192677,5.037031,133.020910,1.384462,10.088872,1.898112,8.929873,682.836708,24.522126,104.156406,10.887271,56.894530,16.471871,0.437286,0.670168,13.920097,2.812067,122.773379,2.786186,2.792722,5.916082,5.770425,17.643437,3.244145,25.172442,23.273834,0.074384
min,1.000000,15.000000,10.000000,12.500000,1.000000,0.200000,21.000000,3.300000,0.00000,-29.000000,1.000000,34.000000,0.200000,0.100000,0.100000,10.000000,5.000000,0.000000,2.000000,24.000000,15.000000,20.000000,0.500000,1.300000,20.000000,1.000000,12.000000,20.000000,0.010000,66.000000,9.400000,23.000000,0.100000,0.010000,21.000000,6.820000
25%,7879.000000,52.000000,28.500000,27.800000,12.000000,1.400000,36.000000,9.200000,21.00000,-3.000000,15.000000,177.000000,2.800000,7.600000,0.700000,36.000000,21.000000,0.400000,137.000000,95.000000,108.000000,71.000000,1.700000,3.700000,54.250000,7.300000,53.000000,96.000000,0.100000,103.000000,27.300000,72.000000,0.500000,0.030000,105.000000,7.330000
50%,15726.000000,64.000000,33.000000,32.200000,17.000000,2.100000,37.000000,10.500000,23.90000,-1.000000,18.000000,233.000000,3.400000,10.400000,0.900000,40.000000,36.000000,0.500000,189.000000,97.000000,130.000000,80.000000,2.000000,4.100000,62.000000,8.200000,72.000000,98.000000,0.300000,107.000000,30.900000,83.000000,0.800000,0.150000,119.000000,7.370000
75%,23725.000000,74.000000,38.000000,40.600000,27.000000,3.400000,37.000000,12.000000,26.00000,0.000000,21.000000,316.000000,4.200000,14.000000,1.380000,45.000000,84.000000,0.600000,251.000000,98.000000,160.000000,91.000000,2.200000,4.500000,72.000000,8.700000,104.000000,100.000000,1.210000,110.000000,35.000000,95.000000,1.400000,2.050000,137.000000,7.410000
max,31658.000000,100.000000,100.000000,250.000000,268.000000,31.000000,42.000000,23.800000,50.00000,100.000000,97.000000,1179.000000,16.400000,440.000000,41.900000,100.000000,9961.000000,4000.000000,2322.000000,100.000000,952.000000,300.000000,9.600000,10.750000,298.000000,20.600000,3833.000000,100.000000,21.200000,141.000000,63.400000,191.000000,46.500000,440.000000,287.000000,7.780000


### Features

In [4]:
df_mean = df.groupby("pid").mean().add_suffix("_mean")
df_var = df.groupby("pid").var().add_suffix("_var")
df_min = df.groupby("pid").min().add_suffix("_min")
df_max = df.groupby("pid").max().add_suffix("_max")



In [5]:
df_train = pd.concat([df_mean, df_var, df_min, df_max], axis=1)
df_train.shape


(18995, 140)

In [6]:
df_train.head()


,Age_mean,EtCO2_mean,PTT_mean,BUN_mean,Lactate_mean,Temp_mean,Hgb_mean,HCO3_mean,BaseExcess_mean,RRate_mean,Fibrinogen_mean,Phosphate_mean,WBC_mean,Creatinine_mean,PaCO2_mean,AST_mean,FiO2_mean,Platelets_mean,SaO2_mean,Glucose_mean,ABPm_mean,Magnesium_mean,Potassium_mean,ABPd_mean,Calcium_mean,Alkalinephos_mean,SpO2_mean,Bilirubin_direct_mean,Chloride_mean,Hct_mean,Heartrate_mean,Bilirubin_total_mean,TroponinI_mean,ABPs_mean,pH_mean,Age_var,EtCO2_var,PTT_var,BUN_var,Lactate_var,Temp_var,Hgb_var,HCO3_var,BaseExcess_var,RRate_var,Fibrinogen_var,Phosphate_var,WBC_var,Creatinine_var,PaCO2_var,AST_var,FiO2_var,Platelets_var,SaO2_var,Glucose_var,ABPm_var,Magnesium_var,Potassium_var,ABPd_var,Calcium_var,...,Fibrinogen_min,Phosphate_min,WBC_min,Creatinine_min,PaCO2_min,AST_min,FiO2_min,Platelets_min,SaO2_min,Glucose_min,ABPm_min,Magnesium_min,Potassium_min,ABPd_min,Calcium_min,Alkalinephos_min,SpO2_min,Bilirubin_direct_min,Chloride_min,Hct_min,Heartrate_min,Bilirubin_total_min,TroponinI_min,ABPs_min,pH_min,Age_max,EtCO2_max,PTT_max,BUN_max,Lactate_max,Temp_max,Hgb_max,HCO3_max,BaseExcess_max,RRate_max,Fibrinogen_max,Phosphate_max,WBC_max,Creatinine_max,PaCO2_max,AST_max,FiO2_max,Platelets_max,SaO2_max,Glucose_max,ABPm_max,Magnesium_max,Potassium_max,ABPd_max,Calcium_max,Alkalinephos_max,SpO2_max,Bilirubin_direct_max,Chloride_max,Hct_max,Heartrate_max,Bilirubin_total_max,TroponinI_max,ABPs_max,pH_max
pid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,34.0,NaN,NaN,12.0,NaN,36.750000,8.566667,25.333333,-0.666667,17.000000,NaN,4.6,5.233333,0.50,43.333333,NaN,0.425000,143.0,NaN,120.000000,68.333333,1.80,4.000000,50.250000,7.6,NaN,100.000000,NaN,112.0,23.20,77.083333,NaN,NaN,114.500000,7.370000,0.0,NaN,NaN,0.0,NaN,0.500000,0.013333,1.333333,1.066667,3.272727,NaN,NaN,0.853333,NaN,2.333333,NaN,0.002500,NaN,NaN,NaN,69.151515,0.135,0.030000,50.431818,NaN,...,NaN,4.6,4.7,0.50,42.0,NaN,0.4,143.0,NaN,120.0,53.0,1.2,3.8,39.0,7.6,NaN,100.0,NaN,111.0,22.2,58.0,NaN,NaN,100.0,7.33,34.0,NaN,NaN,12.0,NaN,38.0,8.7,26.0,0.0,18.0,NaN,4.6,6.3,0.50,45.0,NaN,0.5,143.0,NaN,120.0,84.0,2.1,4.1,62.5,7.6,NaN,100.0,NaN,114.0,24.6,99.0,NaN,NaN,142.0,7.41
2,86.0,NaN,31.8,32.0,NaN,36.000000,13.100000,NaN,NaN,18.000000,NaN,3.8,9.300000,2.12,NaN,NaN,NaN,226.0,NaN,111.000000,94.636364,2.10,3.800000,70.090909,9.5,NaN,96.000000,NaN,NaN,40.20,59.000000,NaN,0.44,131.181818,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,23.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,800.000000,40.454545,NaN,NaN,43.290909,NaN,...,NaN,3.8,9.3,2.12,NaN,NaN,NaN,226.0,NaN,91.0,84.0,2.1,3.8,58.0,9.5,NaN,94.0,NaN,NaN,40.2,59.0,NaN,0.44,120.0,NaN,86.0,NaN,31.8,32.0,NaN,36.0,13.1,NaN,NaN,24.0,NaN,3.8,9.3,2.12,NaN,NaN,NaN,226.0,NaN,131.0,106.0,2.1,3.8,79.0,9.5,NaN,98.0,NaN,NaN,40.2,59.0,NaN,0.44,148.0,NaN
4,66.0,NaN,34.6,8.0,NaN,36.666667,10.550000,NaN,NaN,14.636364,NaN,3.1,7.500000,0.53,NaN,34.0,NaN,269.0,NaN,100.166667,80.909091,2.45,3.500000,49.600000,8.2,130.0,99.272727,0.1,NaN,33.55,72.545455,0.6,0.02,125.454545,NaN,0.0,NaN,NaN,0.0,NaN,0.333333,0.405000,NaN,NaN,23.054545,NaN,NaN,2.420000,0.0128,NaN,NaN,NaN,200.0,NaN,590.583333,934.690909,0.845,0.080000,75.377778,0.08,...,NaN,3.1,6.4,0.45,NaN,34.0,NaN,259.0,NaN,75.0,56.0,1.8,3.3,40.0,8.0,130.0,96.0,0.1,NaN,32.0,68.0,0.6,0.02,86.0,NaN,66.0,NaN,34.6,8.0,NaN,37.0,11.0,NaN,NaN,22.0,NaN,3.1,8.6,0.61,NaN,34.0,NaN,279.0,NaN,123.5,164.0,3.1,3.7,70.0,8.4,130.0,100.0,0.1,NaN,35.1,78.0,0.6,0.02,168.0,NaN
6,66.0,NaN,53.8,32.0,1.8,37.166667,10.316667,19.500000,-2.857143,15.833333,NaN,3.1,39.700000,1.35,39.000000,NaN,0.566667,105.0,95.0,113.500000,65.750000,3.10,4.633333,48.833333,7.8,NaN,99.333333,NaN,113.5,30.39,87.333333,NaN,NaN,100.666667,7.352857,0.0,NaN,NaN,0.0,0.0,1.424242,0.393667,0.500000,1.809524,20.515152,NaN,NaN,NaN,0.0050,4.666667,NaN,0.050667,NaN,56.333333,1727.750000,20.750000,NaN,0.102667,12.060606,NaN,...,NaN,3.1,39.7,1.30,36.0,NaN,0.4,105.0,78.0,62.0,60.0,3.1,4.2,42.0,7.8,N

In [7]:
df_train.describe()


,Age_mean,EtCO2_mean,PTT_mean,BUN_mean,Lactate_mean,Temp_mean,Hgb_mean,HCO3_mean,BaseExcess_mean,RRate_mean,Fibrinogen_mean,Phosphate_mean,WBC_mean,Creatinine_mean,PaCO2_mean,AST_mean,FiO2_mean,Platelets_mean,SaO2_mean,Glucose_mean,ABPm_mean,Magnesium_mean,Potassium_mean,ABPd_mean,Calcium_mean,Alkalinephos_mean,SpO2_mean,Bilirubin_direct_mean,Chloride_mean,Hct_mean,Heartrate_mean,Bilirubin_total_mean,TroponinI_mean,ABPs_mean,pH_mean,Age_var,EtCO2_var,PTT_var,BUN_var,Lactate_var,Temp_var,Hgb_var,HCO3_var,BaseExcess_var,RRate_var,Fibrinogen_var,Phosphate_var,WBC_var,Creatinine_var,PaCO2_var,AST_var,FiO2_var,Platelets_var,SaO2_var,Glucose_var,ABPm_var,Magnesium_var,Potassium_var,ABPd_var,Calcium_var,...,Fibrinogen_min,Phosphate_min,WBC_min,Creatinine_min,PaCO2_min,AST_min,FiO2_min,Platelets_min,SaO2_min,Glucose_min,ABPm_min,Magnesium_min,Potassium_min,ABPd_min,Calcium_min,Alkalinephos_min,SpO2_min,Bilirubin_direct_min,Chloride_min,Hct_min,Heartrate_min,Bilirubin_total_min,TroponinI_min,ABPs_min,pH_min,Age_max,EtCO2_max,PTT_max,BUN_max,Lactate_max,Temp_max,Hgb_max,HCO3_max,BaseExcess_max,RRate_max,Fibrinogen_max,Phosphate_max,WBC_max,Creatinine_max,PaCO2_max,AST_max,FiO2_max,Platelets_max,SaO2_max,Glucose_max,ABPm_max,Magnesium_max,Potassium_max,ABPd_max,Calcium_max,Alkalinephos_max,SpO2_max,Bilirubin_direct_max,Chloride_max,Hct_max,Heartrate_max,Bilirubin_total_max,TroponinI_max,ABPs_max,pH_max
count,18995.000000,1322.000000,7816.000000,13966.000000,4872.000000,18552.000000,13975.000000,7837.000000,5697.000000,18843.000000,1714.000000,9329.000000,13341.000000,13727.000000,7772.000000,4796.000000,7678.000000,13541.000000,5055.000000,16330.000000,18888.000000,12326.000000,14997.000000,15113.000000,12266.000000,4744.000000,18973.000000,623.000000,8351.000000,14639.000000,18988.000000,4711.000000,2792.000000,18597.000000,7994.000000,18995.0,1241.000000,1965.000000,4665.000000,2607.000000,17492.000000,5399.000000,3258.000000,4493.000000,18715.000000,536.000000,1985.000000,4271.000000,3603.000000,5178.000000,8.340000e+02,6.059000e+03,3715.000000,3085.000000,10608.000000,18809.000000,4010.000000,7297.000000,14990.000000,3769.000000,...,1714.000000,9329.000000,13341.000000,13727.000000,7772.000000,4796.000000,7678.000000,13541.000000,5055.000000,16330.000000,18888.000000,12326.000000,14997.000000,15113.000000,12266.000000,4744.000000,18973.000000,623.000000,8351.000000,14639.000000,18988.000000,4711.000000,2792.000000,18597.000000,7994.000000,18995.000000,1322.000000,7816.000000,13966.000000,4872.000000,18552.000000,13975.000000,7837.000000,5697.000000,18843.000000,1714.000000,9329.000000,13341.000000,13727.000000,7772.000000,4796.000000,7678.000000,13541.000000,5055.000000,16330.000000,18888.000000,12326.000000,14997.000000,15113.000000,12266.000000,4744.000000,18973.000000,623.000000,8351.000000,14639.000000,18988.000000,4711.000000,2792.000000,18597.000000,7994.000000
mean,62.073809,32.829817,38.801443,23.000573,2.383041,36.774057,10.748832,23.801594,-0.789992,18.126663,272.937922,3.618139,11.561759,1.469968,40.940753,158.859112,0.671290,210.708276,93.310435,138.766495,82.389830,1.995272,4.113012,64.319841,7.738113,96.675671,97.609473,1.170195,105.925903,32.005177,84.337059,1.544023,6.414359,122.605389,7.375355,0.0,14.860326,435.053950,8.856027,1.095161,0.331599,0.762110,2.438686,3.862145,11.083155,3594.850169,0.493290,9.413715,0.104841,28.439539,1.034008e+05,6.600705e+02,718.084133,68.583706,1524.911097,98.903146,0.118694,0.194963,66.783213,5.688269,...,263.747025,3.543102,11.225502,1.446586,38.145876,149.685884,0.468945,206.993354,89.719881,122.292983,69.753812,1.945789,3.982462,54.513375,7.406501,95.865620,94.833869,1.136998,105.509101,31.243130,75.067148,1.515390,5.858800,104.271764,7.347282,62.073809,36.686838,40.852719,23.354683,2.734101,37.150172,10.940312,24.119376,0.333193,22.925702,281.853092,3.695353,11.912339,1.493806,44.000116,168.243641,1.186276,214.492393,95.548764,156.428317,97.476864,2.044613,4.248312,76.593082,8.1635

In [8]:
from sklearn.impute import SimpleImputer  # todo: try KNNImputer

imputer = SimpleImputer(strategy="median")
imputer.fit(df_train)


SimpleImputer(strategy='median')

In [9]:
X_train = imputer.transform(df_train)


In [10]:
 np.isnan(X_train).sum()

0

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)


StandardScaler()

In [12]:
df_training_labels = pd.read_csv("../train_labels.csv")

TESTS = [
    "LABEL_BaseExcess",
    "LABEL_Fibrinogen",
    "LABEL_AST",
    "LABEL_Alkalinephos",
    "LABEL_Bilirubin_total",
    "LABEL_Lactate",
    "LABEL_TroponinI",
    "LABEL_SaO2",
    "LABEL_Bilirubin_direct",
    "LABEL_EtCO2",
]
y_train = df_training_labels[TESTS]


### Models

In [13]:
# from sklearn.pipeline import make_pipeline
# from sklearn.svm import SVC

# clf = make_pipeline(StandardScaler(), SVC())
# clf.fit(X_train, y_train_task1[:, 0])


In [14]:
# clf.predict(X_train)


In [15]:
from sklearn.ensemble import HistGradientBoostingClassifier

params = {
    "loss": "binary_crossentropy",
    "l2_regularization": 0.01,
    "early_stopping": True,
    "scoring": "roc_auc",
}

baseline_HGB = HistGradientBoostingClassifier(**params)
baseline_HGB.fit(X_train, y_train.iloc[:, 0])



HistGradientBoostingClassifier(early_stopping=True, l2_regularization=0.01,
                               loss='binary_crossentropy', scoring='roc_auc')

In [16]:
baseline_HGB.score(X_train, y_train.iloc[:, 0])



0.731824164253751

In [17]:
clf = HistGradientBoostingClassifier(**params)

for i in TESTS:
    clf.fit(X_train, y_train[i])
    print(clf.score(X_train, y_train[i]))


0.7337720452750723
0.9262963937878389
0.760357988944459
0.7639905238220585
0.7622532245327718
0.7997894182679652
0.9002369044485391
0.7663069228744407
0.966254277441432
0.9339826270071071


In [18]:
clf.fit(X_train, df_training_labels["LABEL_Sepsis"])
clf.score(X_train, df_training_labels["LABEL_Sepsis"])



0.9428270597525664

In [19]:
from sklearn.ensemble import HistGradientBoostingRegressor

params = {
    "loss": "squared_error",
    "early_stopping": True,
    "scoring": "r2",
    "max_depth": 3,
}

reg = HistGradientBoostingRegressor(**params)

for i in range(1, 5):
    reg.fit(X_train, y_train.iloc[:, -i])
    print(reg.score(X_train, y_train.iloc[:, -i]))



0.01084856700379655
0.006347799493811701
0.005577023067031672
0.007380743193605377


In [20]:
from sklearn.linear_model import Ridge

lm = Ridge()

for i in range(1, 5):
    lm.fit(X_train, y_train.iloc[:, -i])
    print(0.5 + 0.5 * np.maximum(0, lm.score(X_train, y_train.iloc[:, -i])))



0.5045599689919058
0.5032895066205731
0.5035197949915249
0.5037788987820353


In [21]:
# cross-validation for hyperparameter selection
# from sklearn.experimental import enable_halving_search_cv
# from sklearn.model_selection import HalvingGridSearchCV


# params = {
#     "loss": "binary_crossentropy",
#     "early_stopping": True,
#     "scoring": "roc_auc",
# }

# clf = HistGradientBoostingClassifier(**params)

# param_grid = {
#     "learning_rate": [0.1, 0.5, 0.7, 1.0],
#     "max_depth": [3, None],
#     "l2_regularization": [0.01, 0.1, 1, 5],
# }

# search = HalvingGridSearchCV(clf, param_grid, resource='n_samples',).fit(X_train, y_train.iloc[:,0])

In [22]:
# cross-validation for hyperparameter selection
# from sklearn.model_selection import GridSearchCV


# params = {
#     "loss": "binary_crossentropy",
#     "early_stopping": True,
#     "scoring": "roc_auc",
# }

# clf = HistGradientBoostingClassifier(**params)

# param_grid = {
#     "learning_rate": [0.1, 0.2, 0.5],
#     "max_depth": [3, None],
#     "l2_regularization": [0, 0.01, 0.1, 1, 5],
# }

# search = GridSearchCV(clf, param_grid).fit(X_train, y_train[TESTS[2]])

In [23]:
# search.best_score_

In [24]:
# search.cv_results_['mean_test_score'], search.cv_results_['params']

### KNNImputer

In [25]:
# from sklearn.impute import KNNImputer

# imputer = KNNImputer()
# imputer.fit(df_train)

# X_train = imputer.transform(df_train)

In [26]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)

params = {
    "loss": "binary_crossentropy",
    "l2_regularization": 0.01,
    "early_stopping": True,
    "scoring": "roc_auc",
}

clf = HistGradientBoostingClassifier(**params)

for i in TESTS:
    clf.fit(X_train, y_train[i])
    print(clf.score(X_train, y_train[i]))

0.7326138457488813
0.9262963937878389
0.7611476704395894
0.7637799420900236
0.75951566201632
0.8001579362990261
0.9003421953145565
0.766412213740458
0.966096341142406
0.9340352724401159


In [27]:
df_test = pd.read_csv('../test_features.csv')

In [28]:
def get_features(df):
    df.drop("Time", axis=1, inplace=True)
    df_mean = df.groupby("pid").mean().add_suffix("_mean")
    df_var = df.groupby("pid").var().add_suffix("_var")
    df_min = df.groupby("pid").min().add_suffix("_min")
    df_max = df.groupby("pid").max().add_suffix("_max")

    return pd.concat([df_mean, df_var, df_min, df_max], axis=1)

In [29]:
df_test_features = get_features(df_test)
df_test_features.head()

,Age_mean,EtCO2_mean,PTT_mean,BUN_mean,Lactate_mean,Temp_mean,Hgb_mean,HCO3_mean,BaseExcess_mean,RRate_mean,Fibrinogen_mean,Phosphate_mean,WBC_mean,Creatinine_mean,PaCO2_mean,AST_mean,FiO2_mean,Platelets_mean,SaO2_mean,Glucose_mean,ABPm_mean,Magnesium_mean,Potassium_mean,ABPd_mean,Calcium_mean,Alkalinephos_mean,SpO2_mean,Bilirubin_direct_mean,Chloride_mean,Hct_mean,Heartrate_mean,Bilirubin_total_mean,TroponinI_mean,ABPs_mean,pH_mean,Age_var,EtCO2_var,PTT_var,BUN_var,Lactate_var,Temp_var,Hgb_var,HCO3_var,BaseExcess_var,RRate_var,Fibrinogen_var,Phosphate_var,WBC_var,Creatinine_var,PaCO2_var,AST_var,FiO2_var,Platelets_var,SaO2_var,Glucose_var,ABPm_var,Magnesium_var,Potassium_var,ABPd_var,Calcium_var,...,Fibrinogen_min,Phosphate_min,WBC_min,Creatinine_min,PaCO2_min,AST_min,FiO2_min,Platelets_min,SaO2_min,Glucose_min,ABPm_min,Magnesium_min,Potassium_min,ABPd_min,Calcium_min,Alkalinephos_min,SpO2_min,Bilirubin_direct_min,Chloride_min,Hct_min,Heartrate_min,Bilirubin_total_min,TroponinI_min,ABPs_min,pH_min,Age_max,EtCO2_max,PTT_max,BUN_max,Lactate_max,Temp_max,Hgb_max,HCO3_max,BaseExcess_max,RRate_max,Fibrinogen_max,Phosphate_max,WBC_max,Creatinine_max,PaCO2_max,AST_max,FiO2_max,Platelets_max,SaO2_max,Glucose_max,ABPm_max,Magnesium_max,Potassium_max,ABPd_max,Calcium_max,Alkalinephos_max,SpO2_max,Bilirubin_direct_max,Chloride_max,Hct_max,Heartrate_max,Bilirubin_total_max,TroponinI_max,ABPs_max,pH_max
pid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,39.0,NaN,41.350000,18.500000,NaN,36.0,9.65,14.500000,-6.5,11.714286,147.000000,5.40,18.000,2.300000,28.0,31.5,0.53,193.50,NaN,217.500000,80.750000,2.35,4.150000,64.500000,8.15,114.000000,100.000000,NaN,100.000000,28.45,84.500000,24.100000,NaN,128.625000,7.37,0.0,NaN,16.245000,4.500000,NaN,0.000000,0.605,4.500000,12.5,0.904762,NaN,0.720,0.500000,0.020000,8.0,0.5,0.002,0.500000,NaN,6160.500000,33.642857,0.045,0.405000,55.071429,0.045,...,147.0,4.8,17.5,2.2,26.0,31.0,0.5,193.0,NaN,162.0,74.0,2.2,3.7,55.0,8.0,109.0,100.0,NaN,98.0,25.9,78.0,21.8,NaN,119.0,7.34,39.0,NaN,44.2,20.0,NaN,36.0,10.2,16.0,-4.0,13.0,147.0,6.0,18.5,2.4,30.0,32.0,0.6,194.0,NaN,273.0,89.0,2.5,4.6,76.0,8.3,119.0,100.0,NaN,102.0,31.00,90.0,26.4,NaN,144.0,7.4
3,84.0,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,16.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.500000,NaN,NaN,74.333333,NaN,NaN,97.000000,NaN,NaN,NaN,99.833333,NaN,NaN,113.000000,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,890.300000,NaN,NaN,563.866667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,NaN,NaN,57.0,NaN,NaN,94.0,NaN,NaN,NaN,90.0,NaN,NaN,82.0,NaN,84.0,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147.0,NaN,NaN,121.0,NaN,NaN,100.0,NaN,NaN,NaN,110.0,NaN,NaN,180.0,NaN
5,62.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,18.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.400000,NaN,NaN,49.500000,NaN,NaN,95.909091,NaN,NaN,NaN,65.272727,NaN,NaN,106.800000,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,3.254545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.044444,NaN,NaN,8.944444,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,NaN,NaN,46.0,NaN,NaN,92.0,NaN,NaN,NaN,62.0,NaN,NaN,90.0,NaN,62.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,NaN,NaN,56.0,NaN,NaN,98.0,NaN,NaN,NaN,71.0,NaN,NaN,127.0,NaN
7,71.0,NaN,75.766667,21.333333,NaN,36.4,8.20,24.666667,0.5,16.181818,177.333333,5.15,6.525,1.233333,43.0,1792.0,0.50,98.75,70.0,234.333333,88.090909,1.95,4.733333,66.909091,9.00,113.666667,99.636364,1.25,100.666667,22.99,98.272727,2.033333,NaN,129.181818,7.38,0.0,NaN,4142.863333,6.333333,NaN,0.266667,2.260,2.333333,4.5,0.963636,146.333333,0.605,1.149167,0.023333,8.0,169981.0,0.000,62.916667,NaN,764.266667,120.090909,0.125,0.003333,56.290909,0.500,...,168.0,4.6,5.7,1.1,41.0,1547.0,0.5,89.0,70.0,183.0,69.0,1.7,4.7,56.0,8.5,101.0,99.0,1.0,98.0,17.8,88.0,1.7,NaN,94.0,7.36,71.0,NaN,150.0,24.0,NaN

In [30]:
df_pred = pd.DataFrame(columns=df_training_labels.columns)
df_pred['pid'] = df_test_features.index

In [31]:
# %%capture

from sklearn.pipeline import make_pipeline
from tqdm import tqdm

TESTS = [
    "LABEL_BaseExcess",
    "LABEL_Fibrinogen",
    "LABEL_AST",
    "LABEL_Alkalinephos",
    "LABEL_Bilirubin_total",
    "LABEL_Lactate",
    "LABEL_TroponinI",
    "LABEL_SaO2",
    "LABEL_Bilirubin_direct",
    "LABEL_EtCO2",
]

y_train = df_training_labels[TESTS]
X_test = imputer.transform(df_test_features)
X_test = scaler.transform(X_test)

X_train = scaler.transform(X_train)

params = {
    "loss": "binary_crossentropy",
    "l2_regularization": 0.01,
    "early_stopping": True,
    "scoring": "roc_auc",
}

HGB = make_pipeline(HistGradientBoostingClassifier(**params))


for target in tqdm(TESTS):
    HGB.fit(X_train, y_train[target])
    df_pred[target] = HGB.predict_proba(X_test)[:,1]

100%|██████████| 10/10 [00:07<00:00,  1.32it/s]


In [32]:
df_pred.head()

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,0,0.231759,0.061147,0.195127,0.243348,0.214861,0.203728,0.086107,0.217466,0.031360,0.101220,NaN,NaN,NaN,NaN,NaN
1,3,0.314196,0.067716,0.304154,0.244905,0.230591,0.222393,0.094327,0.237231,0.033424,0.071397,NaN,NaN,NaN,NaN,NaN
2,5,0.247207,0.069091,0.237660,0.235963,0.235754,0.196269,0.088656,0.222433,0.030640,0.057887,NaN,NaN,NaN,NaN,NaN
3,7,0.278622,0.047916,0.325533,0.280664,0.262870,0.193437,0.093534,0.213364,0.043669,0.054026,NaN,NaN,NaN,NaN,NaN
4,9,0.240657,0.068371,0.248915,0.229940,0.229113,0.298590,0.127265,0.230185,0.026665,0.078247,NaN,NaN,NaN,NaN,NaN


In [33]:
HGB.fit(X_train, df_training_labels["LABEL_Sepsis"])
df_pred["LABEL_Sepsis"] = HGB.predict_proba(X_test)[:, 1]